In [8]:
import pandas as pd
import numpy as np
import csv
import gc
import os
import copy
from sklearn.model_selection import train_test_split
from random import randint

In [4]:
directory='../GeneInteractionsBN_Datasets/Batches'
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
# Dat Size and proportions
#files = pd.DataFrame(np.random.randn(10, 5), columns=['a', 'b', 'c', 'd', 'e'])
folder = 'Tt'
directory = '../GeneInteractionsBN_Datasets/Labeled/'
for filename_same in os.listdir(directory):
    data_list = {}
    total_size = 0
    censored_listdir = copy.deepcopy(os.listdir(directory))
    print(censored_listdir)
    censored_listdir.pop(censored_listdir.index(filename_same)) # Censoring one for testing
    save_folder = '../GeneInteractionsBN_Datasets/Batches/'+'censored_'+filename_same.split('.')[0]+'/'
    for filename in censored_listdir:
        if filename.endswith(".csv"): 
            print(os.path.join(directory, filename))
            csv = pd.read_csv(filepath_or_buffer=os.path.join(directory, filename),sep=',')
            key_name = filename.split('.')[0].split('_')[0]
            data_list[key_name]=[0,[]]
            #print(csv['Target'][0])
            for i in range(0,len(csv)):
                value = int(csv['Target'][i])
                if (value==1 or value==0):
                    # data_list[key_name][1][0] for index
                    # data_list[key_name][1][2] for marker
                    data_list[key_name][1].append([i,value,0])
                    data_list[key_name][0]=data_list[key_name][0]+1
            #print(key_name)
            #print(data_list[key_name])
            total_size = total_size + data_list[key_name][0]
            print(data_list[key_name][0])
            print(total_size)
            del csv
            gc.collect()
            continue
        else:
            continue
    extract_directory = '../GeneInteractionsBN_Datasets/DataGeneCausality/'+folder+'/'
    batch_size = 6000
    n_loops = int(total_size/batch_size)
    for i in range(0,n_loops):
        csv_output = []
        for filename in os.listdir(extract_directory):
            if filename.endswith(".csv"):
                print(os.path.join(extract_directory, filename))
                csv = pd.read_csv(filepath_or_buffer=os.path.join(extract_directory, filename),sep=';')
                pd_headers = csv.columns.insert(0,'TrueIndex')
                key_name = filename.split('.')[0]
                try:
                    contribution_size = int(data_list[key_name][0]*batch_size/total_size)
                except:
                    contribution_size = 0
                print(contribution_size)
                while contribution_size>0:
                    chosen_index = randint(0, data_list[key_name][0]-1)
                    # data_list[key_name][1][i][0] for index
                    # data_list[key_name][1][i][2] for marker
                    if data_list[key_name][1][chosen_index][2] == 0: # proceeds
                        data_list[key_name][1][chosen_index][2] = 1 # sets marker
                        contribution_size = contribution_size-1
                        # inserting first the true index and then the values
                        csv_output.append(np.insert(csv.iloc[data_list[key_name][1][chosen_index][0],:].values,0,data_list[key_name][1][chosen_index][0]))
                del csv
                gc.collect()
                continue
            else:
                continue
        df = pd.DataFrame(data=csv_output,columns=pd_headers)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        df.to_csv(save_folder+'batch_'+str(i)+'.csv')
        del csv_output
        del df
    del data_list
    gc.collect()

['CTNNB1_Target.csv', 'Erk_Target.csv', 'Ikk2_Target.csv', 'IRF4_Target.csv', 'Jnk_Target.csv', 'LEF1_Target.csv', 'MYC_Target.csv']
../GeneInteractionsBN_Datasets/Labeled/Erk_Target.csv
140730
140730
../GeneInteractionsBN_Datasets/Labeled/Ikk2_Target.csv
108984
249714
../GeneInteractionsBN_Datasets/Labeled/IRF4_Target.csv
96654
346368
../GeneInteractionsBN_Datasets/Labeled/Jnk_Target.csv
213102
559470
../GeneInteractionsBN_Datasets/Labeled/LEF1_Target.csv
68070
627540
../GeneInteractionsBN_Datasets/Labeled/MYC_Target.csv
64002
691542
../GeneInteractionsBN_Datasets/DataGeneCausality/Tt/CTNNB1.csv
0
../GeneInteractionsBN_Datasets/DataGeneCausality/Tt/Erk.csv
1221
../GeneInteractionsBN_Datasets/DataGeneCausality/Tt/Ikk2.csv
945
../GeneInteractionsBN_Datasets/DataGeneCausality/Tt/IRF4.csv
838
../GeneInteractionsBN_Datasets/DataGeneCausality/Tt/Jnk.csv
1848
../GeneInteractionsBN_Datasets/DataGeneCausality/Tt/LEF1.csv
590
../GeneInteractionsBN_Datasets/DataGeneCausality/Tt/MYC.csv
555
../Ge

../GeneInteractionsBN_Datasets/DataGeneCausality/Tt/MYC.csv
555
../GeneInteractionsBN_Datasets/DataGeneCausality/Tt/CTNNB1.csv
0
../GeneInteractionsBN_Datasets/DataGeneCausality/Tt/Erk.csv
1221
../GeneInteractionsBN_Datasets/DataGeneCausality/Tt/Ikk2.csv


In [65]:
marked = 0
for key in data_list:
    for i in data_list[key][1]:
        if i[2]==1:
            marked=marked+1
print(marked)

0


In [ ]:
directory = '../GeneInteractionsBN_Datasets/DataGeneCausality/Raw/'

batch_size = 6000
n_loops = int(total_size/batch_size)
for i in range(0,n_loops):
    csv_output = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            print(os.path.join(directory, filename))
            csv = pd.read_csv(filepath_or_buffer=os.path.join(directory, filename),sep=';')
            pd_headers = csv.columns.insert(0,'TrueIndex')
            key_name = filename.split('.')[0]
            contribution_size = int(data_list[key_name][0]*batch_size/total_size)
            print(contribution_size)
            while contribution_size>0:
                chosen_index = randint(0, data_list[key_name][0]-1)
                # data_list[key_name][1][i][0] for index
                # data_list[key_name][1][i][2] for marker
                if data_list[key_name][1][chosen_index][2] == 0: # proceeds
                    data_list[key_name][1][chosen_index][2] = 1 # sets marker
                    contribution_size = contribution_size-1
                    # inserting first the true index and then the values
                    csv_output.append(np.insert(csv.iloc[data_list[key_name][1][chosen_index][0],:].values,0,data_list[key_name][1][chosen_index][0]))
            del csv
            gc.collect()
            continue
        else:
            continue
    df = pd.DataFrame(data=csv_output,columns=pd_headers)
    df.to_csv('../GeneInteractionsBN_Datasets/Batches/'+'batch_'+str(i)+'.csv')
    del csv_output
    del df


../GeneInteractionsBN_Datasets/DataGeneCausality/Raw/LEF1.csv
527
../GeneInteractionsBN_Datasets/DataGeneCausality/Raw/MYC.csv
496
../GeneInteractionsBN_Datasets/DataGeneCausality/Raw/CTNNB1.csv
636
../GeneInteractionsBN_Datasets/DataGeneCausality/Raw/Erk.csv
1091
../GeneInteractionsBN_Datasets/DataGeneCausality/Raw/IRF4.csv
749
../GeneInteractionsBN_Datasets/DataGeneCausality/Raw/Ikk2.csv
845
../GeneInteractionsBN_Datasets/DataGeneCausality/Raw/Jnk.csv
1652
../GeneInteractionsBN_Datasets/DataGeneCausality/Raw/LEF1.csv
527
../GeneInteractionsBN_Datasets/DataGeneCausality/Raw/MYC.csv


In [56]:
marked = 0
for key in data_list:
    for i in data_list[key][1]:
        if i[2]==1:
            marked=marked+1
print(marked)

767488


In [57]:
n_loops

128